In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import itertools
import networkx as nx
import ast
import math

In [2]:
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return (sum_x/length, sum_y/length, sum_z/length)

In [3]:
def split(word):
    return [char for char in word]

In [4]:
def getDistances(list1):
    distances_list1 = {}
    for p1 in list1:
        index_str = '[' + str(p1[0]) +','+ str(p1[1]) +','+ str(p1[2]) + ']'
        distances_list1[index_str] = []
        for p2 in list1:
                distances_list1[index_str].append(math.dist(p1,p2))
    return distances_list1

In [23]:
def getOverlap(list1,list2):
    list1 = getDistances(list1)
    list2 = getDistances(list2)
    result = []
    for d1 in list1:
        maxOverlap = 0
        maxOverlapIndex = None 
        for d2 in list2:
            setA = set(list1[d1])
            setB = set(list2[d2])
            overlap = setA & setB
            result1 = float(len(overlap)) / len(setA) * 100
            if result1 > maxOverlap:
                maxOverlap = result1 
                maxOverlapIndex = d2
        result.append((d1,maxOverlapIndex,maxOverlap))
    result.sort(key=lambda i:i[2],reverse=True)    
    cnt = 0
    score = result[0][2]
    for ele in result:
        if ele[2] == score:
            cnt += 1
    if cnt == 12:
        return result[:12]
    else:
        return -1

In [6]:
def getOrigin(list1,list2):
    overlap = getOverlap((list1),(list2))
    overScan1 = [ast.literal_eval(x[0]) for x in overlap]
    overScan2 = [ast.literal_eval(x[1]) for x in overlap]
    center1 = centeroidnp(np.array(overScan1))
    center2 = centeroidnp(np.array(overScan2))
    #print(center1,center2)
    for ele in overScan1:
        ele[0] -= center1[0]
        ele[1] -= center1[1]
        ele[2] -= center1[2]
    for ele in overScan2:
        ele[0] -= center2[0]
        ele[1] -= center2[1]
        ele[2] -= center2[2]    

    P = np.matrix([[x[0] for x in overScan1],[x[1] for x in overScan1],[x[2] for x in overScan1]]).T
    Q = np.matrix([[x[0] for x in overScan2],[x[1] for x in overScan2],[x[2] for x in overScan2]]).T
    #print(P.shape,Q.shape)
    H = P.T * Q 
    u, s, vh = np.linalg.svd(H, full_matrices=True)
    v = vh.T
    d = np.linalg.det(v @ u.T)
    e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])
    r = v @ e @ u.T
    t = ast.literal_eval(overlap[1][0]) - ast.literal_eval(overlap[1][1]) * r
    return ([0,0,0]*r+t,r,t)

In [62]:
# Using readlines()
file1 = open('aoc-19-ex', 'r')
file1 = open('aoc-19', 'r')
Lines = file1.readlines()
Lines = [x.replace('\n','') for x in Lines]
scanners = []
helper = []
for ele in Lines:
    if ele.find('scanner') > 0 :
        scanners.append(helper)
        helper = []
    else:
        xo = ele.split(',')
        helper.append([int(xo[0]),int(xo[1]),int(xo[2])])
scanners.append(helper)
scanners.pop(0)

[]

In [63]:
# get scanner ovelap
while len(scanners) > 1:
    scannerToMatch = None
    for s1 in scanners[1:]:
        if getOverlap(scanners[0],s1) != -1:
            print('scanner found',scanners.index(s1))
            scannerToMatch = s1.copy()
            scanners.remove(s1)
            break
    #print(scannerToMatch)
    p1,s1_r,s1_t = getOrigin(scanners[0],scannerToMatch)
    print('scanner at: ', [0,0,0]*s1_r+s1_t)
    helper = []
    for ele in scannerToMatch:
        xo = ele*s1_r+s1_t
        if not np.round(xo).tolist()[0] in scanners[0]:
            helper.append(np.round(xo).tolist()[0])
    scanners[0] += helper
print(len(scanners[0]))

scanner found 3
scanner at:  [[  77. -115. 1203.]]
scanner found 1
scanner at:  [[1242. -114. 1281.]]
scanner found 4
scanner at:  [[-1037.   -79.  1249.]]
scanner found 10
scanner at:  [[   21. -1353.  1367.]]
scanner found 5
scanner at:  [[  134. -2389.  1296.]]
scanner found 7
scanner at:  [[   85. -1257.  2529.]]
scanner found 8
scanner at:  [[  150.   -54. -1133.]]
scanner found 7
scanner at:  [[-1.60e+01 -1.00e+00 -2.29e+03]]
scanner found 5
scanner at:  [[   49. -1358. -2423.]]
scanner found 2
scanner at:  [[ 1312. -1365. -2268.]]
scanner found 8
scanner at:  [[ 1225. -2488.  1310.]]
scanner found 10
scanner at:  [[ 135. 1083. 1298.]]
scanner found 1
scanner at:  [[1260. 1127.  130.]]
scanner found 2
scanner at:  [[  15. 1071. 2503.]]
scanner found 5
scanner at:  [[-2.000e+00 -5.800e+01  3.593e+03]]
scanner found 11
scanner at:  [[-1160.  -130.  3771.]]
scanner found 11
scanner at:  [[-2345.     4.  1359.]]
scanner found 11
scanner at:  [[ 1203. -1191. -3577.]]


TypeError: 'NoneType' object is not iterable

In [51]:
p1,s1_r,s1_t = getOrigin(scanners[0],scannerToMatch)

In [32]:
p1,s1_r,s1_t = getOrigin(scanners[0],scanners[1])
print('scanner 1 at: ', [0,0,0]*s1_r+s1_t,len(scanners[0]))
helper = []
for ele in scanners[1]:
    xo = ele*s1_r+s1_t
    if not np.round(xo).tolist()[0] in scanners[0]:
        helper.append(np.round(xo).tolist()[0])
scanners[0] += helper

p4,s4_r,s4_t = getOrigin(scanners[0],scanners[4])
print('scanner 4 at: ', [0,0,0]*s4_r+s4_t)
helper = []
for ele in scanners[4]:
    xo = ele*s4_r+s4_t
    if not np.round(xo).tolist()[0] in scanners[0]:
        helper.append(np.round(xo).tolist()[0])
scanners[0] += helper

p3,s3_r,s3_t = getOrigin(scanners[0],scanners[3])
print('scanner 3 at: ', [0,0,0]*s3_r+s3_t)
helper = []
for ele in scanners[3]:
    xo = ele*s3_r+s3_t
    if not np.round(xo).tolist()[0] in scanners[0]:
        helper.append(np.round(xo).tolist()[0])
scanners[0] += helper

p2,s2_r,s2_t = getOrigin(scanners[0],scanners[2])
print('scanner 2 at: ', [0,0,0]*s2_r+s2_t)
helper = []
for ele in scanners[2]:
    xo = ele*s2_r+s2_t
    if not np.round(xo).tolist()[0] in scanners[0]:
        helper.append(np.round(xo).tolist()[0])
scanners[0] += helper
print(len(scanners[0]))

scanner 1 at:  [[   68. -1246.   -43.]] 25
scanner 4 at:  [[  -20. -1133.  1061.]]
scanner 3 at:  [[  -92. -2380.   -20.]]
scanner 2 at:  [[ 1105. -1205.  1229.]]
79


79
